### 📌 RunnablePassthrough
- 입력을 가공하지 않고 그대로 전달하는 Runnable
- 주로 **체인 중간 디버깅/테스트** 용도
- 파이프라인 설계할 때 “그대로 흘려보내기” 역할

In [6]:
#  RunnablePassthrough
from langchain.schema.runnable import RunnablePassthrough

# 입력을 그대로 출력
passthrough = RunnablePassthrough()
print(passthrough.invoke("안녕하세요!"))

안녕하세요!


### 📌 Runnable 구조확인
- LCEL에서 `|` 연산자는 **Runnable** 객체를 연결
- 모든 체인은 `RunnableSequence`로 구현됨
- `.input_schema` / `.output_schema` → 입력/출력 타입 확인 가능

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:10]) 

sk-proj-73


In [3]:
# Runnable 구조확인
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
prompt = ChatPromptTemplate.from_template("너의 이름은 무엇이니?")

chain = prompt | llm
print(type(chain))          # RunnableSequence
print(chain.input_schema)   # 입력 구조
print(chain.output_schema)  # 출력 구조

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.utils.pydantic.PromptInput'>
<class 'langchain_openai.chat_models.base.ChatOpenAIOutput'>


### 📌 RunnableLambda
- 파이썬 함수를 Runnable로 감싸서 체인에 포함 가능
- LLM 호출 없이 **로직 추가**할 때 유용

In [4]:
# RunnableLambda
from langchain.schema.runnable import RunnableLambda

# 간단한 함수 → Runnable로 래핑
adder = RunnableLambda(lambda x: x + 10)
print(adder.invoke(5))

15


### 📌 Routing
- RunnableBranch를 사용해 **조건에 따라 체인 분기**
- 예: 수학 질문 → 계산 LLM, 일반 질문 → 대화 LLM
- 복잡한 워크플로우 설계 시 유용

In [7]:
# 사용자의 질문 의도에 따라 Routing
from langchain.schema.runnable import RunnableBranch

llm_math = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def router(input_text: str):
    if any(char.isdigit() for char in input_text):
        return "math"
    return "chat"

branch = RunnableBranch(
    (lambda x: router(x) == "math", llm_math),
    (lambda x: router(x) == "chat", llm_chat),
    # default
    RunnablePassthrough()
)

print(branch.invoke("2 + 2 = ?"))
print(branch.invoke("오늘 기분 어때?"))

content='4' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 13, 'total_tokens': 14, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C9vbHxtFCENWwPXUL55zO3Mx0ilKK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--2139ebcb-79b4-40a2-a9e8-ec5bdc5bb499-0' usage_metadata={'input_tokens': 13, 'output_tokens': 1, 'total_tokens': 14, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='저는 항상 같은 기분이에요. 근데 여러분은 어떠신가요? 행복한 하루 되세요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 17, 'total_tokens': 59, 'completion_tokens_details': {'accepted_p

### 📌 RunnableParallel & itemgetter
- **RunnableParallel**: 여러 Runnable을 병렬 실행 → 결과를 dict로 반환
- **itemgetter**: 입력 dict에서 특정 key 추출
- 병렬 실행으로 **효율성 증가** & 다양한 출력 동시에 생성 가능


In [8]:
#  RunnableParallel, itemgetter
from operator import itemgetter
from langchain.schema.runnable import RunnableParallel

parallel = RunnableParallel(
    first=itemgetter("question"),
    second=lambda x: x["question"].upper()
)

print(parallel.invoke({"question": "오늘 날씨 어때?"}))

{'first': '오늘 날씨 어때?', 'second': '오늘 날씨 어때?'}


### 📌 Config를 활용한 동적 체인 변경
- `RunnableConfig`로 실행 시점에 **LLM, Prompt 등 교체 가능**
- 하나의 체인으로 다양한 스타일/설정을 실험할 수 있음


In [9]:
# 동적으로 LLM 이나 Prompt 를 변경하는 방법(Config)
from langchain.schema.runnable.config import RunnableConfig

prompt1 = ChatPromptTemplate.from_template("너는 친절한 어시스턴트야. {q}")
prompt2 = ChatPromptTemplate.from_template("너는 간단명료한 어시스턴트야. {q}")

chain = prompt1 | llm

# 기본 실행
print(chain.invoke({"q": "LangChain이 뭐야?"}))

# Config로 prompt 변경
config = RunnableConfig()
dynamic_chain = prompt2 | llm
print(dynamic_chain.invoke({"q": "LangChain이 뭐야?"}))

content='LangChain은 블록체인 기술을 활용하여 언어 서비스 및 번역 서비스를 제공하는 플랫폼입니다. 이를 통해 언어 관련 서비스의 투명성과 효율성을 높이고, 사용자들 간의 신뢰를 증진시킬 수 있습니다.LangChain은 사용자들이 안전하고 신뢰할 수 있는 언어 서비스를 이용할 수 있도록 도와줍니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 30, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C9vc1RBUAkbAgoW0zJCBkTGtlm3HI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--2d052cef-fb8e-4a06-84ae-6bf3e0b3a6a6-0' usage_metadata={'input_tokens': 30, 'output_tokens': 143, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='LangChain은 언어 및 통역 서비스를 제공하는 블록체인 기술 기반의 플랫폼입니다. 이 

### 📌 @chain 데코레이터
- 파이썬 함수에 `@chain` 붙이면 Runnable로 변환
- `invoke`, `batch`, `stream` 같은 메서드 사용 가능
- 장점:
  - 직관적인 함수 정의
  - 기존 함수 로직을 그대로 체인에 통합

In [11]:
# # 파이썬 함수에 chain 데코레이터로 runnable 설정
# from langchain.chains import chain

# @chain
# def custom_chain(x: str) -> str:
#     return x[::-1]  # 문자열 뒤집기

# print(custom_chain.invoke("LangChain"))

# ✅ 최종 정리
- **RunnablePassthrough** → 입력 그대로 전달  
- **Runnable 구조확인** → 체인 input/output schema 확인  
- **RunnableLambda** → 함수 기반 runnable  
- **Routing (Branch)** → 조건 분기  
- **RunnableParallel & itemgetter** → 병렬 실행  
- **Config** → 실행 시점에 동적 prompt/LLM 변경  
- **@chain 데코레이터** → 파이썬 함수도 runnable 체인으로 확장  

👉 LCEL 고급 문법은 RAG, Agent, Workflow 구현 시 **유연한 체인 구성**을 가능하게 해줌
